# In another R algoritm I search "workforce mobility" related information on the web and store the new HTML content every day. The R algorithm also cleans the webcontent and takes care that only meaningful content is stored. The result is saved into a csv file and handed over a Python algoritm for furher analysis.

# This Python algoritm make a step further in the preparation of text corpus for mashine learning. Only necessary attributes are used. As a result the original csv file is extended with labels which mark different business categories of the ducuments being listed.


In [5]:
# Read necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

from sklearn.preprocessing import MinMaxScaler


Review the actual work environment

In [6]:
import os
 
dirpath = os.getcwd()
print("current directory is : " + dirpath)
foldername = os.path.basename(dirpath)
print("Directory name is : " + foldername)

current directory is : /home/csepeli_csaba/VEKOP
Directory name is : VEKOP


In [7]:
# path="/home/aibpo/presentation"
# os.chdir(path)

In [8]:
# Reading the csv format text file that was prepared by R algorithm 

# Importing the dataset
dataset1 = pd.read_csv('export_htmlcontent_04162019_en.csv',sep=',')
# dataset1 = pd.read_csv('mkb_export.csv',sep=',')
print(dataset1.shape)
dataset1.head(5)


(1563, 3)


,collectdate,daily_url,html_content
0,2018-08-18,https://www.fairfieldcurrent.com/2018/08/18/fi...,l (VDSI). usand blog and news sources in re...
1,2018-11-30,https://www.fayettecountyrecord.com/articles/2...,
2,2018-11-30,https://austinfilmschool.org/calendar/2018/11/...,
3,2018-11-30,https://www.washingtonpost.com/politics/2018/1...,y for severe misconduct. If public employees a...
4,2018-11-30,https://www.federaltimes.com/opinions/2018/11/...,"of public services. Further, young workers of..."


In [9]:
dataset1 = dataset1.drop(dataset1.columns[[1]], axis=1)
print(dataset1.shape)
dataset1.head(5)

(1563, 2)


,collectdate,html_content
0,2018-08-18,l (VDSI). usand blog and news sources in re...
1,2018-11-30,
2,2018-11-30,
3,2018-11-30,y for severe misconduct. If public employees a...
4,2018-11-30,"of public services. Further, young workers of..."


In [10]:
corpus_mwa = dataset1['html_content'].tolist()

In [11]:
len(corpus_mwa)

1563

In [12]:
len(corpus_mwa[14])

1198

In [13]:
corpus_mwa[14]

'tional 1.2K MBoe will be needed, projected to lead to +15% oil demand growth to 2040. Secondly, cost productivity: decarbonising energy supplies to meet tighter regulatory frameworks will put additional pressure on conventional energy, as they need to compete with other sources. And finally, the aging workforce. How can downstream oil & gas quickly adapt to retain knowledge built over several generations? And perhaps more challenging, how can the industry be perceived as exciting and cutting-edge, ready to develop and embrace technology to attract the brightest, most talented people? gh there is a wide variety of digital capability and expertise available, few have successfully cracked embedding digital in an industry that is renowned for its risk adverse behaviour. utions can help wherever you are in your transformation journey. rthern Europe, Martin Carruth, will participate in this event and will provide leadership during a panel discussion on the challenges and opportunities digit

In [14]:
type(corpus_mwa[14]) # checking the type of valiable

str

In [15]:
with open('corpus_mwa', 'wb') as fp:
    pickle.dump(corpus_mwa, fp)

## Prepare the tf-idf matrix for further analyzis, typical basic table format for text mining: sparse matrix. 

In [16]:
# Read document corpus (or sample of it in case if you want to process only part of corpus)

vectorizer = TfidfVectorizer(min_df = 0, max_df=0.5, stop_words = "english", ngram_range = (1,1), max_features = 100)
# vectorizer = TfidfVectorizer(min_df = 0.1, max_df=0.5, ngram_range = (1,1), max_features = 100)

# X will be hte DFM (data frequency matrix)
vectorizer_fitted = vectorizer.fit(corpus_mwa)
X = vectorizer_fitted.transform(corpus_mwa)
vocab = vectorizer.get_feature_names()

### Save the vectorizer model for later use

In [17]:
# save the vectorizer fitted model to disk
filename = 'mwa_vectorizer_fitted_model.sav'
pickle.dump(vectorizer_fitted, open(filename, 'wb'))

In [18]:
X

<1563x100 sparse matrix of type '<class 'numpy.float64'>'
	with 17096 stored elements in Compressed Sparse Row format>

In [19]:
type(X)

scipy.sparse.csr.csr_matrix

In [20]:
type(vocab)

list

In [21]:
vocab[1:10]

['10',
 '2018',
 'based',
 'best',
 'better',
 'board',
 'business',
 'businesses',
 'care']

# Use data.frame instead of sparse matrix, this is better for visual analyzis

In [22]:
dfofterms = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
dfofterms.head(5)

,000,10,2018,based,best,better,board,business,businesses,care,...,value,way,women,work,workers,workforce,working,world,year,years
0,0.0,0.672329,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.352184,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.440725,0.000000,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.150869,0.0,0.0,0.0,...,0.279877,0.0,0.0,0.283198,0.124181,0.099435,0.0,0.0,0.0,0.0


In [23]:
dfofterms.shape

(1563, 100)

# Standard scale the extended dataframe which contanins weights

In [24]:
scaler = MinMaxScaler()

In [25]:
df_scaled = pd.DataFrame(scaler.fit_transform(dfofterms), columns=dfofterms.columns)

In [26]:
df_scaled.head(5)

,000,10,2018,based,best,better,board,business,businesses,care,...,value,way,women,work,workers,workforce,working,world,year,years
0,0.0,0.672329,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.530514,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.440725,0.000000,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.227262,0.0,0.0,0.0,...,0.279877,0.0,0.0,0.283198,0.124181,0.129334,0.0,0.0,0.0,0.0


# Draw dendogram from DFM sparse matrix to find the optimum number of clusters 
code will be inserted here later

# Make clustering with k-means

It is possible to train the clusteing model with data frame

In [27]:
true_k = 4
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
km_model_fitted = model.fit(df_scaled)

Save the kmean clustering model for later use

In [ ]:
# save the model to disk
filename = 'mwa_kmeans_model.sav'
pickle.dump(km_model_fitted, open(filename, 'wb'))

print out the top terms in each cluster

In [28]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
#terms.append('cost')
for i in range(true_k):
    print('\n')
    print(" Cluster %d:" % i ),
    for ind in order_centroids[i, :15]:
           print(' %s ' % terms[ind], end = ''),
    print

Top terms per cluster:


 Cluster 0:
 percent  year  workforce  said  women  new  jobs  employees  years  growth  000  million  workers  economy  time 

 Cluster 1:
 time  com  information  center  group  organization  said  performance  business  large  new  public  best  work  people 

 Cluster 2:
 workforce  work  new  jobs  people  government  health  workers  students  said  state  training  high  better  employees 

 Cluster 3:
 data  company  digital  business  growth  customer  service  technology  global  customers  market  services  new  companies  management 

## Add cluster label to the corpus csv file

In [30]:
dataset2 = pd.read_csv('export_htmlcontent_04162019_en.csv',sep=',')
print(dataset2.shape)
dataset1.head(5)

(1563, 3)


,collectdate,html_content
0,2018-08-18,l (VDSI). usand blog and news sources in re...
1,2018-11-30,
2,2018-11-30,
3,2018-11-30,y for severe misconduct. If public employees a...
4,2018-11-30,"of public services. Further, young workers of..."


In [31]:
# add cluster information to the corpus and write out as a csv file
csv_cluster = pd.concat([dataset2,pd.DataFrame(model.labels_,columns = ['cluster'])],axis=1)

In [32]:
csv_cluster.head(5)

,collectdate,daily_url,html_content,cluster
0,2018-08-18,https://www.fairfieldcurrent.com/2018/08/18/fi...,l (VDSI). usand blog and news sources in re...,1
1,2018-11-30,https://www.fayettecountyrecord.com/articles/2...,,1
2,2018-11-30,https://austinfilmschool.org/calendar/2018/11/...,,1
3,2018-11-30,https://www.washingtonpost.com/politics/2018/1...,y for severe misconduct. If public employees a...,2
4,2018-11-30,https://www.federaltimes.com/opinions/2018/11/...,"of public services. Further, young workers of...",2


In [33]:
#write cluster information to corpus date
csv_cluster.to_csv('corpus_clusters_en.csv',  encoding='utf-8')